In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Creazione del dataset partendo dalla cartella video

Root
- GT (video segmentati)
-- Sunny
--- FL0
--- FL1_2
--- ...

- Color (video a colori)
-- Sunny
--- FL0
--- FL1_2
--- ...

Vogliamo ottenere

SynthOffRoad
- train
-- GT
--- Sunny
---- FL0
----- 00334.png
----- 04422.png
---- ...

-- Color
--- Sunny
---- FL0
----- 00334.png
----- 04422.png
---- ...


## Step 1: Estrazione Frame

In [39]:
video_folder = "/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5"
video_sunny_folder = video_folder + "/Sunny"
video_gt_folder = video_folder + "/GT"

frame_folder = "/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/SynthOffRoad"
frame_sunny_folder = frame_folder + "/Sunny"
frame_gt_folder = frame_folder + "/GT"

Per primo step creiamo una grande unica cartella in cui mettere tutti i frame e poi dividiamo in train valid e test

In [40]:
import cv2
import os

def unpack_video(video_path, output_dir):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Read the video frame by frame
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as an image
        image_path = os.path.join(output_dir, f"{frame_num:06d}.png")
        cv2.imwrite(image_path, frame)

        frame_num += 1

    cap.release()

def invert_grayscale_video(input_path, output_path):
    # Check if the input path exists
    if not os.path.exists(input_path):
        print(f"Error: The input file '{input_path}' does not exist.")
        return

    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{input_path}'")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format

    # Create a VideoWriter object to save the output video
    out = cv2.VideoWriter(output_path, codec, fps, (width, height), isColor=False)

    while True:
        # Read each frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform gray-level inversion
        inverted_frame = 255 - gray_frame

        # Write the inverted frame to the output video
        out.write(inverted_frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Inverted video saved as '{output_path}'")

In [46]:
# Take all the *.avi files in the video folder (there are more levels of folder) with glob

import glob

video_files = glob.glob(video_folder + "/**/*.avi", recursive=True)
video_files

['/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL1.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL4.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL0.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL1_2.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL3.avi',
 '/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/GT/FL4.

In [42]:
for video_path in video_files:
    # Substitue in video_file the video folder with the frame folder
    frame_path = frame_folder + video_path.split(video_folder)[1]
    frame_path = frame_path.split(".")[0]

    print(video_path + " -> " + frame_path)

    if not os.path.exists(frame_path):
        if "inv" in video_path:
            print("Already inverted " + video_path)
        elif "GT" in video_path and "WL" in video_path:
            print("Inverting " + video_path)
            video_path_2 = video_path.replace("WL", "WL_inv")
            invert_grayscale_video(video_path, video_path_2)
            unpack_video(video_path_2, frame_path)
        else:
            unpack_video(video_path, frame_path)
    else:
        print("Skipping " + video_path + ". Folder already exists")

/content/drive/MyDrive/Magistrale/Tesi/Video Scenari/Bozza5/Sunny/WL4.avi -> /content/drive/MyDrive/Magistrale/Tesi/Video Scenari/SynthOffRoad/Sunny/WL4


KeyboardInterrupt: 

In [ ]:
# For each folder in Color, check if the elements inside the folder are the same in the folder under GT with same name

color_folders = glob.glob(frame_sunny_folder + "/*")
print(color_folders)

for color_folder in color_folders:
    gt_folder = color_folder.replace("Sunny", "GT")
    print(color_folder + " -> " + gt_folder)
    color_files = len(os.listdir(color_folder))
    gt_files = len(os.listdir(gt_folder))

    print(f"{color_folder}: {color_files} color files - {gt_files} gt files")


In [ ]:
# Take the frame files under the "Sunny" folder
frame_files = glob.glob(frame_sunny_folder + "/**/*.png", recursive=True)
print("Number of frame extracted: " + str(len(frame_files)))

In [ ]:
train_per, val_per, test_per = 0.7, 0.2, 0.1

In [ ]:
# Create directories for train test and val and move files

import random
import shutil

# Shuffle the list of files
random.shuffle(frame_files)

# Split the list of files using the percentages
train_files = frame_files[:int(len(frame_files)*train_per)]
val_files = frame_files[int(len(frame_files)*train_per):int(len(frame_files)*(train_per+val_per))]
test_files = frame_files[int(len(frame_files)*(train_per+val_per)):]

# Create the directories
train_dir = frame_folder + "/Train"
val_dir = frame_folder + "/Val"
test_dir = frame_folder + "/Test"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(train_dir + "/images")
    os.makedirs(train_dir + "/GT")
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
    os.makedirs(val_dir + "/images")
    os.makedirs(val_dir + "/GT")
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    os.makedirs(test_dir + "/images")
    os.makedirs(test_dir + "/GT")

# Move the files
for color_image in train_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Train/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Train/GT").split("/")[:-1])

    print(image_dest_path)
    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.move(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.move(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")

for color_image in val_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Val/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Val/GT").split("/")[:-1])

    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.move(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.move(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")

for color_image in test_files:
    # Take the name of the parent folder
    image_dest_path = "/".join(color_image.replace("Sunny", "Test/images").split("/")[:-1])
    gt_dest_path = "/".join(color_image.replace("Sunny", "Test/GT").split("/")[:-1])

    print(image_dest_path)
    if not os.path.exists(image_dest_path):
        os.makedirs(image_dest_path)
    if not os.path.exists(gt_dest_path):
        os.makedirs(gt_dest_path)

    # If there is also the corresponding GT file, move it
    try:
        shutil.move(color_image.replace("Sunny", "GT"), gt_dest_path)
        shutil.move(color_image, image_dest_path)
    except FileNotFoundError:
        print("GT file not available for " + color_image + " Skipping...")

In [ ]:
# Count elements in each folder

train_images = glob.glob(train_dir + "/images/**/*.png", recursive=True)
train_gt = glob.glob(train_dir + "/GT/**/*.png", recursive=True)
val_images = glob.glob(val_dir + "/images/**/*.png", recursive=True)
val_gt = glob.glob(val_dir + "/GT/**/*.png", recursive=True)
test_images = glob.glob(test_dir + "/images/**/*.png", recursive=True)
test_gt = glob.glob(test_dir + "/GT/**/*.png", recursive=True)

print(f"Train: {len(train_images)} images - {len(train_gt)} gt")
print(f"Val: {len(val_images)} images - {len(val_gt)} gt")
print(f"Test: {len(test_images)} images - {len(test_gt)} gt")

In [ ]:
import glob
import os

# Remove all .DS_Store files

ds_store_files = glob.glob(frame_folder + "/**/.DS_Store", recursive=True)
print("Num of .DS_Store files found: " + str(len(ds_store_files)))

for ds_store_file in ds_store_files:
    os.remove(ds_store_file)

In [ ]:
!zip -r /content/drive/MyDrive/Magistrale/Tesi/Video\ Scenari/SynthOffRoad.zip /content/drive/MyDrive/Magistrale/Tesi/Video\ Scenari/SynthOffRoad